# Streaks analysis


Streaks analysis is done by [Koch (20004)](https://www.climate-service-center.de/imperia/md/content/gkss/institut_fuer_kuestenforschung/ksd/paper/kochw_ieee_2004.pdf) algorithm implementation.


In [ ]:
# import needed modules
import xsar
import xsarsea
import xsarsea.gradients

import xarray as xr
import numpy as np
import scipy
import os
import time

import logging
logging.basicConfig()
logging.getLogger('xsar.utils').setLevel(logging.DEBUG)
logging.getLogger('xsarsea.streaks').setLevel(logging.DEBUG)

import holoviews as hv
hv.extension('bokeh')
import geoviews as gv
from holoviews.operation.datashader import rasterize


In [ ]:
# open a file a 100m 
filename = xsar.get_test_file('S1A_IW_GRDH_1SDV_20170907T103020_20170907T103045_018268_01EB76_Z010.SAFE') # irma
#filename = xsar.get_test_file('S1B_IW_GRDH_1SDV_20181013T062322_20181013T062347_013130_018428_Z000.SAFE') # bz
filename=xsar.get_test_file('S1B_IW_GRDH_1SDV_20211024T051203_20211024T051228_029273_037E47_Z010.SAFE')
#filename=xsar.get_test_file('S1A_IW_GRDH_1SDV_20170720T112706_20170720T112735_017554_01D5C2_Z010.SAFE') # subswath
sar_ds = xsar.open_dataset(filename,resolution='100m').isel(atrack=slice(20,None,None),xtrack=slice(20,None,None)) # isel to skip bad image edge

# add detrended sigma0
sar_ds['sigma0_detrend'] = xsarsea.sigma0_detrend(sar_ds.sigma0, sar_ds.incidence)
# apply land mask
land_mask = sar_ds['land_mask'].compute()
sar_ds['sigma0_detrend'] = xr.where(land_mask, np.nan, sar_ds['sigma0_detrend']).transpose(*sar_ds['sigma0_detrend'].dims).compute()


## General overview

Gradients direction analysis is done by moving a window over the image. [xsarsea.gradients.Gradients](../basic_api.rst#xsarsea.gradients.Gradients) allow multiple windows sizes and resolutions.

`sar_ds` is a IW_GRDH SAFE with a pixel size of 10m at full resolution. So to compute compute gradients with windows size of 16km and 32km, we need to use `windows_sizes=[1600,3200]`

`sar_ds` resolution is 100m, so if we want to compute gradients at 100m an 200m, we need to use `downscales_factors=[1,2]`

In [ ]:
gradients = xsarsea.gradients.Gradients(sar_ds['sigma0_detrend'].sel(pol='VV'), windows_sizes=[1600,3200], downscales_factors=[1,2])

# get gradients histograms as an xarray dataset
hist = gradients.histogram

# get orthogonals gradients
hist['angles'] = hist['angles'] + np.pi/2

#mean
hist_mean = hist.mean(['downscale_factor','window_size'])

# mean, and smooth
hist_mean_smooth = hist_mean.copy()
hist_mean_smooth['weight'] = xsarsea.gradients.circ_smooth(hist_mean_smooth['weight'])

# smooth only
hist_smooth = hist.copy()
hist_smooth['weight'] = xsarsea.gradients.circ_smooth(hist_smooth['weight'])

# select histogram peak
iangle = hist_mean_smooth['weight'].fillna(0).argmax(dim='angles')
streaks_dir = hist_mean_smooth.angles.isel(angles=iangle)
streaks_weight = hist_mean_smooth['weight'].isel(angles=iangle)
streaks = xr.merge([dict(angle=streaks_dir,weight=streaks_weight)]).drop('angles')


# convert from image convention (rad=0=atrack) to geographic convention (deg=0=north)
# select needed variables in original dataset, and map them to streaks dataset
streaks_geo = sar_ds[['longitude','latitude','ground_heading']].interp(
    atrack=streaks.atrack,
    xtrack=streaks.xtrack, 
    method='nearest')

streaks_geo['weight'] = streaks['weight']

# convert directions from image convention to geographic convetion
# note that there is no clockwise swapping, because image axes are transposed
streaks_geo['streaks_dir'] =  np.rad2deg(streaks['angle']) + streaks_geo['ground_heading']

streaks_geo = streaks_geo.compute()

# plot. Note that hv.VectorField only accept radians, and 0 is West, so we need to reconvert degrees to radians when calling ...
gv.tile_sources.Wikipedia * gv.VectorField(
    (
        streaks_geo['longitude'], 
        streaks_geo['latitude'], 
        np.pi/2 -np.deg2rad(streaks_geo['streaks_dir']), 
        streaks_geo['weight']
    )
).opts(pivot='mid', arrow_heads=False, tools=['hover'], magnitude='Magnitude')



> **_WARNING:_**  `hv.VectorField` and `gv.VectorField` don't use degrees north convention, but radian convention, with 0 = East or right
> So, to use them with degrees north, you have to convert them to gradients with 
> ```python
> np.pi/2 -np.deg2rad(deg_north)
> ```
>

## Digging into intermediate computations 

### streaks_geo

`streaks_geo` is a `xarray.Dataset`, with `latitude`, `longitude` and `streaks_dir` (0=deg north) variables.

It has dims `('atrack', 'xtrack')`, with a spacing corresponding to the first windows size, according to the window step.

In [ ]:
streaks_geo

### streaks

`streaks_geo` was computed from `streaks` (also a `xarray.Dataset`). The main difference is that the `angle` variable from `streaks` is in radians, in *image convention* (ie rad=0 is in atrack direction) 



In [ ]:
streaks

#### Convertion from image convention to geographic convention

```python
angle_geo = np.rad2deg(angle_img) + ground_heading
```

#### Conversion from geographic convention to image convention
```python
angle_img = np.deg2rad(angle_geo - ground_heading)
```



### hist_mean

`streaks` variable was computed from `hist_mean_smooth`.

The main difference with `streaks` variable is that we don't have a single angle, but a histogram of probability for binned angles

In [ ]:
hist_mean_smooth

Let's exctract one histogram at an arbitrary position, and plot the histogram.

We can do this with the regular `hv.Histogram` function, or use [xsarsea.gradients.circ_hist](../basic_api.rst#xsarsea.gradients.circ_hist), that might be used with `hv.Path` to plot the histogram as a circular one.

In [ ]:
hist_at = hist_mean_smooth['weight'].sel(atrack=5000,xtrack=12000,method='nearest')
hv.Histogram( (hist_at.angles, hist_at )) + hv.Path(xsarsea.gradients.circ_hist(hist_at))

`xsarsea` also provide an interactive drawing class [xsarsea.gradients.PlotGradients](../basic_api.rst#xsarsea.gradients.PlotGradients) that can be used to draw the circular histogram at mouse tap. (needs a live notebook)

In [ ]:
# background image for vectorfield
s0 = sar_ds['sigma0_detrend'].sel(pol='VV')
hv_img = rasterize(hv.Image(s0).opts(cmap='gray',clim=(0,np.nanpercentile(s0,95))))


plot_mean_smooth = xsarsea.gradients.PlotGradients(hist_mean_smooth)

# get vectorfield, with mouse tap activated
hv_vf = plot_mean_smooth.vectorfield(tap=True)

# connect mouse to histogram
hv_hist = plot_mean_smooth.mouse_histogram()

# notebook dynamic output
hv_hist + hv_img * hv_vf

`hist_mean_smooth` was smoothed. Let's try `hist_smooth`

In [ ]:
plot_smooth = xsarsea.gradients.PlotGradients(hist_smooth)
hv_vf = plot_smooth.vectorfield()
hv_hist = plot_smooth.mouse_histogram()
hv_hist + (hv_img * hv_vf).opts(legend_position='right', frame_width=300)

Using `source` keyword for `mouse_histogram`, we can link several histrograms

In [ ]:
plot_raw = xsarsea.gradients.PlotGradients(hist)
plot_mean = xsarsea.gradients.PlotGradients(hist_mean)
hv_vf = plot_smooth.vectorfield()

gridspace = hv.GridSpace(kdims=['smooth','mean'])
gridspace[(0,0)] = plot_smooth.mouse_histogram(source=plot_raw)
gridspace[(1,0)] = plot_smooth.mouse_histogram() 
gridspace[(1,1)] = plot_smooth.mouse_histogram(source=plot_mean_smooth) 
gridspace[(0,1)] = plot_smooth.mouse_histogram(source=plot_mean)


gridspace + (hv_img * hv_vf).opts(legend_position='right', frame_width=300)